
# Assignment 3 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 3, PROBLEM 1
Maximum Points = 8


Download the updated data folder from the course github website or just download directly the file [https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/smhi.csv](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/smhi.csv) from the github website and put it inside your data folder, i.e. you want the path `data/smhi.csv`. The data was aquired from SMHI (Swedish Meteorological and Hydrological Institute) and constitutes per hour measurements of wind in the Uppsala Aut station. The data consists of windspeed and direction. Your goal is to load the data and work with it a bit. The code you produce should load the file as it is, please do not alter the file as the autograder will only have access to the original file.

The file information is in Swedish so you need to use some translation service, for instance `Google translate` or ChatGPT.

1. [2p] Load the file, for instance using the `csv` package. Put the wind-direction as a numpy array and the wind-speed as another numpy array.
2. [2p] Use the wind-direction which is an angle in degrees and convert it into a point on the unit circle. Store the `x_coordinate` as one array and the `y_coordinate` as another. From these coordinates, construct the wind-velocity vector.
3. [2p] Calculate the average wind velocity and convert it back to direction and compare it to just taking average of the wind direction as given in the data-file.
4. [2p] The wind velocity is a $2$-dimensional random variable, calculate the empirical covariance matrix which should be a numpy array of shape (2,2).

For you to wonder about, is it more likely for you to have headwind or not when going to the university in the morning.

In [2]:
data = pd.read_csv('data/smhi.csv', sep=';', skiprows=11)
data.head()

,Datum,Tid (UTC),Vindriktning,Kvalitet,Vindhastighet,Kvalitet.1,Unnamed: 6,Tidsutsnitt:
0,2024-06-27,01:00:00,348,G,0.6,G,NaN,Data från senaste fyra månaderna
1,2024-06-27,02:00:00,43,G,0.2,G,NaN,Tidsperiod (fr.o.m.) = 2024-06-27 00:00:01 (UTC)
2,2024-06-27,03:00:00,185,G,0.1,G,NaN,Tidsperiod (t.o.m.) = 2024-11-04 05:00:00 (UTC)
3,2024-06-27,04:00:00,126,G,0.8,G,NaN,Samplingstid = 10 minuter
4,2024-06-27,05:00:00,130,G,0.7,G,NaN,Kvalitetskoderna:


In [1]:
# Import libraries
import pandas as pd 
import numpy as np

# Import the data 
data = pd.read_csv('data/smhi.csv', sep=';', skiprows=11)
# Keep only the non-empty columns
data = data[['Datum','Tid (UTC)', 'Vindriktning', 'Kvalitet', 'Vindhastighet', 'Kvalitet.1','Tidsutsnitt:']]
# In the dataset description it is written that some data has a good quality and some are suspicious - we need to keep only good
# Filter for rows with "Quality" as 'G' (good data)
data = data[data['Kvalitet'] == 'G']

# Separate wind direction and speed data in an array
wind_direction =  data['Vindriktning'].to_numpy(dtype=np.float64)
wind_speed = data['Vindhastighet'].to_numpy(dtype=np.float64)

problem1_wind_direction = wind_direction
problem1_wind_speed = wind_speed



In [2]:

# The wind direction is given as a compass direction in degrees (0-360)
# convert it to x and y coordinates using the standard mathematical convention
# x^2 + y^2 = 1, y=sin(t), x=cos(t)

# Convert degrees to radians
ang_rad = np.radians(wind_direction)

problem1_wind_direction_x_coordinate = np.cos(ang_rad)
problem1_wind_direction_y_coordinate = np.sin(ang_rad)

# Wind-velocity vectors
wind_velocity_x = problem1_wind_direction_x_coordinate * wind_speed
wind_velocity_y = problem1_wind_direction_y_coordinate * wind_speed

problem1_wind_velocity_x_coordinate = wind_velocity_x
problem1_wind_velocity_y_coordinate = wind_velocity_y


In [7]:
# Put the average wind velocity x and y coordinates here in these variables
problem1_average_wind_velocity_x_coordinate = np.mean(wind_velocity_x)
problem1_average_wind_velocity_y_coordinate = np.mean(wind_velocity_y)

# First calculate the angle of the average wind velocity vector in degrees
# Compute the angle (in degrees) of the average vector
problem1_average_wind_velocity_angle_degrees = np.degrees(np.arctan2(problem1_average_wind_velocity_y_coordinate, problem1_average_wind_velocity_x_coordinate)) % 360
# Then calculate the average angle of the wind direction in degrees (using the wind direction in the data)
problem1_average_wind_direction_angle_degrees = np.mean(problem1_wind_direction)  

# 7.09101 is the difference which is too big --> suggest that they are not equal
diff = np.abs(problem1_average_wind_velocity_angle_degrees - problem1_average_wind_direction_angle_degrees)
#diff

# Finally, are they the same? Answer as a boolean value (True or False)
problem1_same_angle = False


In [10]:
wind_velocity_cov =  np.vstack((problem1_wind_velocity_x_coordinate, problem1_wind_velocity_y_coordinate))
covariance_matrix = np.cov(wind_velocity_cov)
#covariance_matrix
problem1_wind_velocity_covariance_matrix = covariance_matrix
# the shapes are as expected now
# print(wind_velocity_cov.shape)  # shape (3061, 2)
# print(problem1_wind_velocity_covariance_matrix.shape) # shape (2,2)
# covariance_matrix

---
## Assignment 3, PROBLEM 2
Maximum Points = 8


For this problem you will need the [pandas](https://pandas.pydata.org/) package and the [sklearn](https://scikit-learn.org/stable/) package. Inside the `data` folder from the course website you will find a file called `indoor_train.csv`, this file includes a bunch of positions in (X,Y,Z) and also a location number. The idea is to assign a room number (Location) to the coordinates (X,Y,Z).

1. [2p] Take the data in the file `indoor_train.csv` and load it using pandas into a dataframe `df_train`
2. [3p] From this dataframe `df_train`, create two numpy arrays, one `Xtrain` and `Ytrain`, they should have sizes `(1154,3)` and `(1154,)` respectively. Their `dtype` should be `float64` and `int64` respectively.
3. [3p] Train a Support Vector Classifier, `sklearn.svc.SVC`, on `Xtrain, Ytrain` with `kernel='linear'` and name the trained model `svc_train`.

To mimic how [kaggle](https://www.kaggle.com/) works, the Autograder has access to a hidden test-set and will test your fitted model.

In [5]:
import pandas as pd 
import numpy as np

df_train = pd.read_csv('data/indoor_train.csv')

df_train = df_train
#df_train.head()

In [6]:
# Rename the variables 
df_train.rename(columns={'Position X': 'X', ' Position Y': 'Y', 'Position Z': 'Z', 'Location': 'Location'}, inplace=True)

# Split the data into dependent and independent vars
Xtrain = df_train[['X', 'Y', 'Z']].to_numpy(dtype=np.float64)
Ytrain = df_train['Location'].to_numpy(dtype=np.int64)

# Check if dimension size is correct 
# Xtrain.shape
# Ytrain.shape

Xtrain = Xtrain
Ytrain = Ytrain


In [7]:
from sklearn.svm import SVC

# Train the SVC model
svc_train = SVC(kernel='linear')
svc_train.fit(Xtrain, Ytrain)
svc_train = svc_train


---
## Assignment 3, PROBLEM 3
Maximum Points = 8


Let us build a proportional model ($\mathbb{P}(Y=1 \mid X) = G(\beta_0+\beta \cdot X)$ where $G$ is the logistic function) for the spam vs not spam data. Here we assume that the features are presence vs not presence of a word, let $X_1,X_2,X_3$ denote the presence (1) or absence (0) of the words $("free", "prize", "win")$.

1. [2p] Load the file `data/spam.csv` and create two numpy arrays, `problem3_X` which has shape **(n_texts,3)** where each feature in `problem3_X` corresponds to $X_1,X_2,X_3$ from above, `problem3_Y` which has shape **(n_texts,)** and consists of a $1$ if the email is spam and $0$ if it is not. Split this data into a train-calibration-test sets where we have the split $40\%$, $20\%$, $40\%$, put this data in the designated variables in the code cell.

2. [2p] Follow the calculation from the lecture notes where we derive the logistic regression and implement the final loss function inside the class `ProportionalSpam`. You can use the `Test` cell to check that it gives the correct value for a test-point.

3. [2p] Train the model `problem3_ps` on the training data. The goal is to calibrate the probabilities output from the model. Start by creating a new variable `problem3_X_pred` (shape `(n_samples,1)`) which consists of the predictions of `problem3_ps` on the calibration dataset. Then train a calibration model using `sklearn.tree.DecisionTreeRegressor`, store this trained model in `problem3_calibrator`. Recall that calibration error is the following for a fixed function $f$
$$
    \sqrt{\mathbb{E}[|\mathbb{E}[Y \mid f(X)] - f(X)|^2]}.
$$

4. [2p] Use the trained model `problem3_ps` and the calibrator `problem3_calibrator` to make final predictions on the testing data, store the prediction in `problem3_final_predictions`. 

In [5]:
# Import libraries
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

# Import the data 
spam = pd.read_csv("data/spam.csv",encoding="latin1")
spam = spam[['v1','v2']]

# Use the words free, prize and win to create binary variables X1, X2, X3 respectively that show if each word is present in every line of emails.
spam["X1"] = spam["v2"].str.contains(r"\bfree\b", case=False, na=False).astype(int)
spam["X2"] = spam["v2"].str.contains(r"\bprize\b", case=False, na=False).astype(int)
spam["X3"] = spam["v2"].str.contains(r"\bwin\b", case=False, na=False).astype(int)

# Independent variables 
problem3_X = spam[["X1", "X2", "X3"]].to_numpy() 

# Dependent variable (spam/not spam)
spam["spam"] = spam["v1"].map({"spam": 1, "ham": 0}).astype(int)
problem3_Y = spam["spam"].to_numpy()  

# 40% train and 60% combined - calibration and test
problem3_X_train, X_comb, problem3_Y_train, Y_comb = train_test_split(problem3_X, problem3_Y, test_size=0.6, stratify=problem3_Y, random_state=10)

# 20% calibration and 40% test
problem3_X_calib, problem3_X_test, problem3_Y_calib, problem3_Y_test = train_test_split(X_comb, Y_comb, test_size=0.6666666666667, stratify=Y_comb, random_state=10)

problem3_X_train = problem3_X_train
problem3_X_calib = problem3_X_calib
problem3_X_test = problem3_X_test

problem3_Y_train = problem3_Y_train
problem3_Y_calib = problem3_Y_calib
problem3_Y_test = problem3_Y_test

# Check if the variables were created correctly

# spam.head()
# Check the shapes of the splits
size_y_train = problem3_Y_train.shape[0]
size_y_calib = problem3_Y_calib.shape[0]
size_y_test = problem3_Y_test.shape[0]

total = size_y_train + size_y_calib + size_y_test
train_perc = size_y_train/total
calib_perc = size_y_calib/total
test_perc = size_y_test/total
#print(train_perc)
#print(calib_perc)
#print(test_perc)
print(problem3_X_train.shape,problem3_X_calib.shape,problem3_X_test.shape,problem3_Y_train.shape,problem3_Y_calib.shape,problem3_Y_test.shape)



(2228, 3) (1114, 3) (2230, 3) (2228,) (1114,) (2230,)


$$ \text{Loss} = -\frac{1}{n} \sum_{i=1}^{n} \left[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right] $$


In [6]:

class ProportionalSpam(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        beta0 = coeffs[0]
        beta = coeffs[1:]
        logit = beta0 + np.dot(X, beta)
        prob = 1 / (1 + np.exp(-logit))
        loss = -np.mean(Y * np.log(prob) + (1 - Y) * np.log(1 - prob))
        return loss
    def fit(self, X, Y):
        import numpy as np
        from scipy import optimize
        
        # Define the optimization objective (loss function) to minimize
        # The 'loss' method of the class is called, which calculates the loss for given coefficients
        opt_loss = lambda coeffs: self.loss(X, Y, coeffs)  # Loss function to optimize

        # Initialize coefficients with zeros (starting point for optimization)
        initial_arguments = np.zeros(shape=X.shape[1] + 1)  # One extra for the intercept term

        # Use scipy's minimize function to optimize the loss function and find the best coefficients
        # The 'cg' method is used here for optimization (Conjugate Gradient)
        self.result = optimize.minimize(opt_loss, initial_arguments, method='cg')

        # Extract the optimized coefficients (including the intercept)
        self.coeffs = self.result.x  # Save the optimized coefficients

    def predict(self, X):
        # Use the trained model (i.e., the learned coefficients) to predict new Y values for input X
        if (self.coeffs is not None):
            # Define the logistic function (sigmoid) for probability prediction
            G = lambda x: np.exp(x) / (1 + np.exp(x))  # Logistic (sigmoid) function

            # Predict using the model coefficients:
            # np.dot(X, self.coeffs[1:]) computes the dot product between X and coefficients (excluding intercept)
            # self.coeffs[0] adds the intercept term
            # The logistic function transforms the result into a probability
            # np.round(..., 10) rounds the probability to 1 decimal place for calibration
            return np.round(10 * G(np.dot(X, self.coeffs[1:]) + self.coeffs[0])) / 10  # Calibrated rounding of the prediction


In [7]:
from sklearn.tree import DecisionTreeRegressor

# Initialize the custom model, for spam prediction
problem3_ps = ProportionalSpam()

# Fit the custom model to the training data (X_train for features, Y_train for labels)
problem3_ps.fit(problem3_X_train, problem3_Y_train)

# Use the fitted model to generate predictions for the calibration data (X_calib)
# The predictions are reshaped into a column vector to match the expected input format for the next step
problem3_X_pred = problem3_ps.predict(problem3_X_calib).reshape(-1, 1)

# Initialize the DecisionTreeRegressor, which will be used for calibrating the initial model's predictions
problem3_calibrator = DecisionTreeRegressor()

# Fit the decision tree regressor to the predictions from the ProportionalSpam model (X_pred) and the actual calibration values (Y_calib)
# The decision tree learns the relationship between the predicted values and the true target labels to calibrate the output
problem3_calibrator.fit(problem3_X_pred, problem3_Y_calib)


DecisionTreeRegressor()

In [8]:
# Use the fitted ProportionalSpam model to make predictions on the test data (X_test)
# The predictions are reshaped into a column vector to match the expected input format for the next step
prob_test_dt = problem3_ps.predict(problem3_X_test).reshape(-1, 1)

# Use the trained decision tree regressor to adjust the predictions from the ProportionalSpam model
# The decision tree has learned how to calibrate these predictions based on the calibration data
problem3_final_predictions = problem3_calibrator.predict(prob_test_dt)



Calibration ensures that the predicted probabilities from a model are aligned with the true probabilities. Here are the steps:


1. **Split the Data**:
   - Divide the training data into two parts:
     - **Training set**: Used to train the initial model.
     - **Calibration set**: Used to train the calibration model.

2. **Train the Initial Model**:
   - Train the base model (e.g., logistic regression, neural network, etc.) on the training set.
   - This model produces uncalibrated probability estimates.

3. **Obtain Predictions on the Calibration Set**:
   - Use the trained base model to predict probabilities for the calibration set.

4. **Train a Calibration Model**:
   - Fit a simple model (calibration model) to adjust the uncalibrated probabilities. Common calibration methods include:
     - **Platt Scaling**: Fits a logistic regression to map the uncalibrated probabilities to calibrated ones.
     - **Isotonic Regression**: A non-parametric method that fits a piecewise constant function.
     - **Decision Trees**: In the given example, a decision tree regressor was trained to calibrate predictions.

5. **Evaluate Calibration**:
   - Use metrics like the **Brier score** or **reliability diagrams** to evaluate the calibrated probabilities.
   - Check if the calibrated probabilities align well with observed outcomes.

6. **Calibrate Predictions on the Test Set**:
   - Use the calibration model to adjust the test predictions generated by the base model.

---
#### Local Test for Assignment 3, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [12]:
try:
    import numpy as np
    test_instance = ProportionalSpam()
    test_loss = test_instance.loss(np.array([[1,0,1],[0,1,1]]),np.array([1,0]),np.array([1.2,0.4,0.3,0.9]))
    assert (np.abs(test_loss-1.2828629432232497) < 1e-6)
    print("Your loss was correct for a test point")
except:
    print("Your loss was not correct on a test point")

Your loss was correct for a test point
